In [97]:
#initialise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.linear_model import LinearRegression
import datetime

In [183]:
#import datasets

recruit = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Average Monthly Recruitment:Resignation Rates by Industry and Occupational Group, Quarterly/average-monthly-recruitment-resignation-rate-by-industry-and-occupational-group-level-3.csv',
                        header=0)

vacancy = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-by-industry-and-occupational-group-level3.csv',
                        header=0)

vacancyr = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-rate-by-industry-and-occupational-group-level3.csv',
                        header=0)

# join recruit, vacancy, vacancyr dataframes and filter for information and comms industry
jobs = vacancy.merge(vacancyr,on=['quarter','industry1','industry2','industry3','occupation1'])[['quarter','industry1','industry2','industry3','occupation1','job_vacancy','job_vacancy_rate']].merge(recruit,on=['quarter','industry1','industry2','industry3','occupation1'])[['quarter','industry1','industry2','industry3','occupation1','job_vacancy','job_vacancy_rate','recruitment_rate','resignation_rate']]

jobs = jobs[jobs['industry2']=='information and communications'].reset_index(drop=True)
jobs[['year', 'quarter']] = jobs['quarter'].str.split('-', 1, expand=True) # split the quarter column into year and quarter columns
jobs.insert(0, 'year', jobs.pop('year')) # move year column to the front


# change the datatypes for all columns in jobs dataset and replace '-' with 0
jobs = jobs.replace('-',0)

convert_dict = {'year':str,
                'quarter':str,
                'industry1':str,
                'industry2':str,
                'industry3':str,
                'occupation1':str,
                'job_vacancy':int,
                'job_vacancy_rate':float,
                'recruitment_rate':float,
                'resignation_rate':float
                }

jobs = jobs.astype(convert_dict)

# drop the rows where numeric data is '0'
jobs = jobs[jobs['year'] != 0][jobs['job_vacancy'] != 0][jobs['job_vacancy_rate'] != 0][jobs['recruitment_rate'] != 0][jobs['resignation_rate'] != 0].reset_index(drop=True)

#view first 10 rows of dataset
jobs.head(10)

/var/folders/nv/c05ywcr140z_fcwc2kx0h69m0000gn/T/ipykernel_36759/3086036398.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  jobs[['year', 'quarter']] = jobs['quarter'].str.split('-', 1, expand=True) # split the quarter column into year and quarter columns
/var/folders/nv/c05ywcr140z_fcwc2kx0h69m0000gn/T/ipykernel_36759/3086036398.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  jobs = jobs[jobs['year'] != 0][jobs['job_vacancy'] != 0][jobs['job_vacancy_rate'] != 0][jobs['recruitment_rate'] != 0][jobs['resignation_rate'] != 0].reset_index(drop=True)


,year,quarter,industry1,industry2,industry3,occupation1,job_vacancy,job_vacancy_rate,recruitment_rate,resignation_rate
0,2000,Q1,services,information and communications,broadcasting and publishing,total,500,6.1,2.7,3.2
1,2000,Q1,services,information and communications,broadcasting and publishing,"professional, managers, executive and technicians",400,8.9,3.2,3.3
2,2000,Q1,services,information and communications,telecommunications,total,700,6.5,2.5,2.0
3,2000,Q1,services,information and communications,telecommunications,"professional, managers, executive and technicians",300,5.1,2.3,1.3
4,2000,Q1,services,information and communications,telecommunications,"clerical, sales and services workers",400,8.4,2.7,2.9
5,2000,Q1,services,information and communications,it and other information services,total,600,4.4,4.3,3.1
6,2000,Q1,services,information and communications,it and other information services,"professional, managers, executive and technicians",600,4.9,4.5,3.2
7,2000,Q2,services,information and communications,broadcasting and publishing,total,400,6.3,3.4,2.7
8,2000,Q2,services,information and communications,broadcasting and publishing,"professional, managers, executive and technicians",400,7.9,3.4,2.7
9,2000,Q2,services,information and communications,broadcasting and publishing,"clerical, sales and services workers",100,3.0,3.6,2.1
